## Code for Creating SUMMA Forcing Files
* Taken from, at some point, Bart's code for creating SUMMA forcing files / ncdfs into the correct format 

In [72]:
# import packages 
# %matplotlib widget
%matplotlib inline

# plotting packages 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

# interactive plotting
"""
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots # adding for subplots
import plotly.figure_factory as ff
"""
# data packages 
# pandas is the most popular Python data analysis/manipulation package, which is built upon NumPy.
import pandas as pd
import numpy as np
import xarray as xr
# import datetime class
# an aware object can locate itself relative to other aware objects. An aware object represents a specific moment in time that is not open to interpretation.
# A naive object does not contain enough information to unambiguously locate itself relative to other date/time objects.
from datetime import datetime

import csv 
import copy 
import os.path 

## About DataFrames
A Spark DataFrame is an immutable set of objects organized into columns and distributed across nodes in a cluster. DataFrames are a SparkSQL data abstraction and are similar to relational database tables or Python Pandas DataFrames. The first step is to import csv into Pandas DataFrame.

## About Pandas

pandas has two main data structures: Series and DataFrame.

Series: a 1-dimensional labeled array that can hold any data type such as integers, strings, floating points, or Python objects. It has row/axis labels as the index.

DataFrame: a 2-dimensional labeled data structure with columns of potentially different types.
It also contains row labels as the index.


In [73]:
# Order doesnt matter
# Load csv into Python as pandas DataFrame. Anything that is a number is int type and anything
# thats NAN is object type
ds =  pd.read_csv("/Users/ianwhidden/pysumma/NLDAS2_WY22.csv")

# the data below is the metadata from the NLDAS website
# 61:APCP:Precipitation hourly total [kg/m^2] Hourly backward accumulated
# 205:DLWRF:Longwave radiation flux downwards (surface) [W/m^2]
# 204:DSWRF:Shortwave radiation flux downwards (surface) [W/m^2]
# 1:PRES:Surface pressure [Pa]
# 51:SPFH:2-m above ground Specific humidity [kg/kg]
# 11:TMP:2-m above ground Temperature [K]
# 33:UGRD:10-m above ground Zonal wind speed [m/s]
#zonal wind is positive if it blows from the west and negative if from the east.
# 34:VGRD:10-m above ground Meridional wind speed [m/s]
#meridional wind is positive if from the south, and negative if from the north


# remove extra columns in NLDAS data
del ds["id"]
del ds[".geo"]

# Convert C to Kelvin
def c_to_f(x):
    return x + 273.15
c = ds["temperature"].map(c_to_f)
ds.temperature = c

ds.head()

# Print row  values of second and third using a list of indexes:
# print(ds.loc[[2,3,4]])



system:index   latitude  longitude  longwave_radiation  pressure  \
0             0  44.207218 -122.11945              362.76  92359.62   
1             1  44.207218 -122.11945              362.81  92384.62   
2             2  44.207218 -122.11945              362.86  92409.52   
3             3  44.207218 -122.11945              356.85  92434.32   
4             4  44.207218 -122.11945              356.87  92456.54   

   shortwave_radiation  specific_humidity  temperature  time_convert  \
0              114.504           0.008863   291.080039  10/1/21 0:00   
1               51.908           0.009004   289.900039  10/1/21 1:00   
2                0.000           0.009145   288.710000  10/1/21 2:00   
3                0.000           0.009287   287.530000  10/1/21 3:00   
4                0.000           0.009220   287.060000  10/1/21 4:00   

           time  total_precipitation  wind_u  wind_v  windspeed  
0  1.630000e+12                  0.0    1.73   -3.00   3.463077  
1  1.630000e+12                  0.0    1.16   -2.75   2.984644  
2  1.630000e+12                  0.0    0.58   -2.51   2.576141  
3  1.630000e+12                  0.0    0.00   -2.26   2.260000  
4  1.630000e+12                  0.0   -0.04   -2.22   2.220360

In [74]:
# Pandas DatetimeIndex.date attribute outputs an
# index object containing the date values present in each of the entries of the DatetimeIndex.
# Im fairly certain this just changes the index to the column of the datetime object.


# Format should be 'Year-Month-Day HR:MIN:SEC'

ds.time = pd.DatetimeIndex(ds['time_convert'], name='time')

# ds.time = pd.to_datetime(ds.time)
ds = ds.drop(columns=['time_convert'])

ds.head()

# The timestamp is Unix timestamp and has 13 numbers

system:index   latitude  longitude  longwave_radiation  pressure  \
0             0  44.207218 -122.11945              362.76  92359.62   
1             1  44.207218 -122.11945              362.81  92384.62   
2             2  44.207218 -122.11945              362.86  92409.52   
3             3  44.207218 -122.11945              356.85  92434.32   
4             4  44.207218 -122.11945              356.87  92456.54   

   shortwave_radiation  specific_humidity  temperature                time  \
0              114.504           0.008863   291.080039 2021-10-01 00:00:00   
1               51.908           0.009004   289.900039 2021-10-01 01:00:00   
2                0.000           0.009145   288.710000 2021-10-01 02:00:00   
3                0.000           0.009287   287.530000 2021-10-01 03:00:00   
4                0.000           0.009220   287.060000 2021-10-01 04:00:00   

   total_precipitation  wind_u  wind_v  windspeed  
0                  0.0    1.73   -3.00   3.463077  
1                  0.0    1.16   -2.75   2.984644  
2                  0.0    0.58   -2.51   2.576141  
3                  0.0    0.00   -2.26   2.260000  
4                  0.0   -0.04   -2.22   2.220360

In [75]:
print(ds.time)

0      2021-10-01 00:00:00
1      2021-10-01 01:00:00
2      2021-10-01 02:00:00
3      2021-10-01 03:00:00
4      2021-10-01 04:00:00
               ...        
8731   2022-09-29 19:00:00
8732   2022-09-29 20:00:00
8733   2022-09-29 21:00:00
8734   2022-09-29 22:00:00
8735   2022-09-29 23:00:00
Name: time, Length: 8736, dtype: datetime64[ns]


In [76]:
#print(ds.time_convert)

In [77]:
# set variable to columns in dataframe
latitude = ds.latitude
longitude = ds.longitude
shortwave = ds.shortwave_radiation
longwave = ds.longwave_radiation
precip = ds.total_precipitation
pressure = ds.pressure
airtemp = ds.temperature
windspd = ds.windspeed
# spechum = niwot_spechum
spechum = ds.specific_humidity
# The actual time and date gets converted into the milliseconds, and it follows the Unix time January 1st, 1970.
# Because it is the date when the time for the Unix computer started.
# The NLDAS data starts at 00:00 GMT on Friday, October 1, 2021 and
# ends at Thu Sep 29 2022 23:00:00 GMT+0000
timestamp = ds.time

In [78]:
# examine contents of dataset
ds.index
ds.columns
ds.dtypes

system:index                    int64
latitude                      float64
longitude                     float64
longwave_radiation            float64
pressure                      float64
shortwave_radiation           float64
specific_humidity             float64
temperature                   float64
time                   datetime64[ns]
total_precipitation           float64
wind_u                        float64
wind_v                        float64
windspeed                     float64
dtype: object

In [79]:
# USING BARTS OLD CODE FOR CREATING 
# Attributes in the netCDF file, with their units and the full length name of the variable 
attrs = {
   'hruId':  {'units': '', 'long_name': 'HRU'},
   'latitude':  {'units': '', 'long_name': 'Latitude'},
   'longitude':  {'units': '', 'long_name': 'Longitude'},
   'data_step':  {'units': '', 'long_name': 'Data Step'},
   'airpres':  {'units': 'Pa', 'long_name': 'Air pressure'},
   'airtemp':  {'units': 'K', 'long_name': 'Air temperature'},
   'spechum':  {'units': 'g g-1', 'long_name': 'Specific humidity'},
   'windspd':  {'units': 'Wind speed', 'long_name': 'm s-1'},
   'SWRadAtm': {'units': 'W m-2', 'long_name': 'Downward shortwave radiation'},
   'LWRadAtm': {'units': 'W m-2', 'long_name': 'Downward longwave radiation'},
   'pptrate':  {'units': 'kg m-2 s-1', 'long_name': 'Precipitation rate'}}

# Latitude, longitude, and elevation (m) for UPLMET in the Andrews
lats = [44.2072180256268] 
lons = [-122.119450090239]
elev = 1050

# Need to better understand what the below lines do. 
bounds = ds.time

# create shape for numpy arrary
shape = (len(bounds), 1, )

# set dimensions of array
dims = ('time', 'hru', )

# set up a dictionary-like container for coordinates in array
coords = {'time': bounds}

# Time stepping
# Forcing timestep units: The user can specify the time units as <units> since <reference time>,
# where <units> is one of seconds, minutes, hours, or days and <reference time> is specified as YYYY-MM-DD hh:mm.
met_data = xr.Dataset(coords=coords)
met_data.time.encoding['calendar'] = 'standard'
met_data.time.encoding['units'] = 'hours since 2021-10-01'

# The variables in SUMMA that are in the forcings
summa_vars = ['airpres', 'airtemp', 'spechum', 
              'windspd', 'SWRadAtm', 'LWRadAtm', 'pptrate']

# From the docs: DataArray provides a wrapper around numpy ndarrays that uses labeled dimensions and coordinates to
# support metadata aware operations. The API is similar to that for the pandas Series or DataFrame, but DataArray objects
# can have any number of dimensions, and their contents have fixed data types.

#xarray.DataArrary(data, coords=None, dims=None, name=None, attrs=None, encoding=None, fastpath=False)
# returns N-dimensional array with labeled coordinates and dimensions.
# DataArray provides a wrapper around numpy ndarrays 
for varname in summa_vars:

#numpy.full(shape, fill_value, dtype=None, order='C', *, like=None)[source]
# Return a new array of given shape and type, filled with fill_value.
    met_data[varname] = xr.DataArray(data=np.full(shape, np.nan),
                                     coords=coords, dims=dims,
                                     name=varname, attrs=attrs[varname])

In [80]:
# DataFrame.loc[source] - Access a group of rows and columns by label(s) or a boolean array.

# Set met_data arraRY with variables and hru ID
#met_data['hruId'].loc[{'hru': 0}] = hruID  
#met_data['latitude'].loc[{'hru': 1}] = latitude  
#met_data['longitude'].loc[{'hru': 1}] = longitude  
met_data['airpres' ].loc[{'hru': 0}] = pressure      #air pressure [Pa]
met_data['airtemp' ].loc[{'hru': 0}] = airtemp       #temperature [K]
met_data['windspd' ].loc[{'hru': 0}] = windspd       #wind speed [m s-1]
met_data['SWRadAtm'].loc[{'hru': 0}] = shortwave     #shortwave [W m-2]
met_data['LWRadAtm'].loc[{'hru': 0}] = longwave      #longwave [w m-2]
met_data['pptrate' ].loc[{'hru': 0}] = precip        #precip [kg m-2 s-1]
met_data['spechum' ].loc[{'hru': 0}] = spechum       #specific humidity [g/g-1]


# For the model run at the Andrews I believe that I need to create the local_attributes netCDF for the forcings.
# In the below code I use the Local Attributes from the reynolds experiment. The plan is to edit the 'snow_zLocalAttributes.nc'
# to contain appropriate info for my site.

# The below uses attributes from the snow_zlocal_attributes netCDF, instead of creating a new local attributes NetCDF for my forcing
# So, using hruID, lat, long, and data_step from a previously constructed NetCDF. 
# I do not have a local attribute NetCDF file set up for the Andrews.


# NetCDF groups are not supported as part of the Dataset data model. Instead, groups can be loaded individually as Dataset
# objects. To do so, pass a group keyword argument to the open_dataset() function.
# The group can be specified as a path-like string, e.g., to access subgroup 'bar' within group 'foo' pass '/foo/bar' as the group argument.
# In a similar way, the group keyword argument can be given to the Dataset.to_netcdf() method to write to a group in a netCDF file. When writing multiple groups in one file, pass mode='a' to Dataset.to_netcdf() to ensure that each call does not delete the file.

# About NetCDF and Xarrary: "Xarray is based on the netCDF data model, so netCDF files on disk directly
# correspond to Dataset objects (more accurately, a group in a netCDF file directly
# corresponds to a Dataset object.)
# Use attributes from the local_attributes netCDF instead of recreating them for the forcings. It may be best to recreate, rather than using reynolds.

ds_local_attrs = xr.open_dataset('/Users/ianwhidden/pysumma/tutorial/Andrews_data/Andrews_test/settings/snow_zLocalAttributes.nc')
#lsosmet = xr.open_dataset('/Users/ianwhidden/local_attributes.nc')
"""
met_data['hruId'] = ds_local_attrs['hruId']
# Lat and long fields don't need to be included in forcing data
met_data['latitude'] = ds_local_attrs['latitude']
met_data['longitude'] = ds_local_attrs['longitude']
"""
# Latitude, longitude UPLMET in the Andrews
#lats = [44.2072180256268] 
#ons = [-122.119450090239]

# The data_step variable is the length of a timestep of the forcing data in seconds.
# 3600 seconds in a hr. This variable should be a data variable, rather than a coordinate.

#met_data['data_step'] = [3600.]
met_data['data_step'] = xr.Variable([], 3600.0)
#met_data['data_step'] = xr.Variable(3600.0, 's')
#met_data['data_step'] = xr.Variable([],np.datetime64(3600,'s')
#met_data['data_step'] = [np.datetime64(3600,'s')]

# Write dataset contents to a netCDF file
met_data.to_netcdf('/Users/ianwhidden/pysumma/tutorial/Andrews_data/Andrews_test/forcing/nldas2022_netCDF.nc')


met_data

<xarray.DataArray 'data_step' ()>
array(3600.)


<xarray.Dataset>
Dimensions:    (time: 8736, hru: 1)
Coordinates:
  * time       (time) datetime64[ns] 2021-10-01 ... 2022-09-29T23:00:00
Dimensions without coordinates: hru
Data variables:
    airpres    (time, hru) float64 9.236e+04 9.238e+04 ... 9.241e+04 9.24e+04
    airtemp    (time, hru) float64 291.1 289.9 288.7 287.5 ... 289.3 289.2 289.1
    spechum    (time, hru) float64 0.008863 0.009004 ... 0.008456 0.008265
    windspd    (time, hru) float64 3.463 2.985 2.576 2.26 ... 2.349 1.91 1.863
    SWRadAtm   (time, hru) float64 114.5 51.91 0.0 0.0 ... 491.8 429.3 340.2
    LWRadAtm   (time, hru) float64 362.8 362.8 362.9 356.9 ... 339.8 339.8 339.8
    pptrate    (time, hru) float64 0.0 0.0 0.0 0.0 ... 0.0336 0.0974 0.0334
    data_step  float64 3.6e+03

In [81]:
np.datetime64(3600,'s')

numpy.datetime64('1970-01-01T01:00:00')

In [82]:
# view contents of snow_zLocalAttributes.nc
#print(lsosmet)
#print(met_data)

#ds_local_attrs

In [83]:
# Code below for another way to view contents of NetCDF. 

"""
import netCDF4 as nc
fn = '/Users/ianwhidden/pysumma/tutorial/Andrews_data/Andrews_test/settings/snow_zLocalAttributes.nc'
snow_zLocalAttributes = nc.Dataset(fn)
snow_zLocalAttributes
"""

"\nimport netCDF4 as nc\nfn = '/Users/ianwhidden/pysumma/tutorial/Andrews_data/Andrews_test/settings/snow_zLocalAttributes.nc'\nsnow_zLocalAttributes = nc.Dataset(fn)\nsnow_zLocalAttributes\n"